In [2]:
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Dense, Embedding
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
import numpy as np


2023-03-29 22:25:51.504420: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
vocab_size = 5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)


In [4]:
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


In [5]:
# Getting all the words from word_index dictionary
word_idx = imdb.get_word_index()

# Originally the index number of a value and not a key,
# hence converting the index as key and the words as values
word_idx = {index: word for word, index in word_idx.items()}

# again printing the review
print([word_idx[i] for i in x_train[0]])


['the', 'as', 'you', 'with', 'out', 'themselves', 'powerful', 'lets', 'loves', 'their', 'becomes', 'reaching', 'had', 'journalist', 'of', 'lot', 'from', 'anyone', 'to', 'have', 'after', 'out', 'atmosphere', 'never', 'more', 'room', 'and', 'it', 'so', 'heart', 'shows', 'to', 'years', 'of', 'every', 'never', 'going', 'and', 'help', 'moments', 'or', 'of', 'every', 'chest', 'visual', 'movie', 'except', 'her', 'was', 'several', 'of', 'enough', 'more', 'with', 'is', 'now', 'current', 'film', 'as', 'you', 'of', 'mine', 'potentially', 'unfortunately', 'of', 'you', 'than', 'him', 'that', 'with', 'out', 'themselves', 'her', 'get', 'for', 'was', 'camp', 'of', 'you', 'movie', 'sometimes', 'movie', 'that', 'with', 'scary', 'but', 'and', 'to', 'story', 'wonderful', 'that', 'in', 'seeing', 'in', 'character', 'to', 'of', '70s', 'and', 'with', 'heart', 'had', 'shadows', 'they', 'of', 'here', 'that', 'with', 'her', 'serious', 'to', 'have', 'does', 'when', 'from', 'why', 'what', 'have', 'critics', 'they'

In [6]:
# Get the minimum and the maximum length of reviews
print("Max length of review:: ", len(max((x_train+x_test), key=len)))
print("Min length of a review:: ", len(min((x_train+x_test), key=len)))


Max length of review::  2697
Min length of a review::  70


In [8]:
from tensorflow.keras.preprocessing import sequence

# Keeping a fixed length of all reviews to max 400 words
max_words = 400
x_train = sequence.pad_sequences(x_train, maxlen=max_words)
x_test = sequence.pad_sequences(x_test, maxlen=max_words)

# Splitting the training set into train and validation sets
x_valid, y_valid = x_train[:641], y_train[:641]
x_train, y_train = x_train[641:], y_train[641:]
embd_len = 32

In [12]:
# fixin every word's embedding size to be 32
embd_len = 32

# Creating a RNN model
RNN_model = Sequential(name="Simple_RNN")
RNN_model.add(Embedding(vocab_size, embd_len, input_length=max_words))
# In case of a stacked (more than one layer of RNN), use return_sequences=True
RNN_model.add(SimpleRNN(128, activation="tanh", return_sequences=False))
RNN_model.add(Dense(1, activation="sigmoid"))

# printing model summary
print(RNN_model.summary())

# Compiling model
RNN_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

# Training the model
history = RNN_model.fit(x_train, y_train, batch_size=64, epochs=5, verbose=1, validation_data=(x_valid, y_valid))

# Printing model score on test data
print()
print("Simple RNN score: ", RNN_model.evaluate(x_test, y_test, verbose=1))


Model: "Simple_RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 400, 32)           160000    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               20608     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 180,737
Trainable params: 180,737
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
381/381 [==============================] - 62s 160ms/step - loss: 0.6870 - accuracy: 0.5403 - val_loss: 0.6830 - val_accuracy: 0.5351
Epoch 2/5
381/381 [==============================] - 106s 279ms/step - loss: 0.5868 - accuracy: 0.6803 - val_loss: 0.6381 - val_accuracy: 0.6427
Epoch 3/5
381/381 [=

In [14]:
# Defining GRU model
gru_model = Sequential(name="GRU_Model")
gru_model.add(Embedding(vocab_size, embd_len, input_length=max_words))
gru_model.add(GRU(128, activation='tanh', return_sequences=False))
gru_model.add(Dense(1, activation='sigmoid'))
# Printing the Summary
print(gru_model.summary())
# Compiling the model
gru_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

# Training the model
history = gru_model.fit(x_train, y_train, batch_size=64, epochs=5, verbose=1, validation_data=(x_valid, y_valid))

# Printing model score on test data
print()
print("Simple GRU score: ", gru_model.evaluate(x_test, y_test, verbose=1))


Model: "GRU_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 400, 32)           160000    
                                                                 
 gru_1 (GRU)                 (None, 128)               62208     
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 222,337
Trainable params: 222,337
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5


2023-03-25 19:19:48.166645: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-25 19:19:48.169178: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-25 19:19:48.170768: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

381/381 [==============================] - ETA: 0s - loss: 0.5187 - accuracy: 0.7272

2023-03-25 19:23:17.063052: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-25 19:23:17.066745: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-25 19:23:17.073248: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

381/381 [==============================] - 211s 546ms/step - loss: 0.5187 - accuracy: 0.7272 - val_loss: 0.3113 - val_accuracy: 0.8736
Epoch 2/5
381/381 [==============================] - 237s 622ms/step - loss: 0.3141 - accuracy: 0.8701 - val_loss: 0.3331 - val_accuracy: 0.8487
Epoch 3/5
381/381 [==============================] - 249s 653ms/step - loss: 0.2506 - accuracy: 0.9006 - val_loss: 0.2590 - val_accuracy: 0.8908
Epoch 4/5
381/381 [==============================] - 310s 813ms/step - loss: 0.2107 - accuracy: 0.9182 - val_loss: 0.2539 - val_accuracy: 0.8970
Epoch 5/5
381/381 [==============================] - 378s 989ms/step - loss: 0.1749 - accuracy: 0.9351 - val_loss: 0.2685 - val_accuracy: 0.8955

782/782 [==============================] - 99s 127ms/step - loss: 0.3429 - accuracy: 0.8755
Simple GRU score:  [0.3429167568683624, 0.8754799962043762]


In [10]:
# Defining LSTM model
lstm_model = Sequential(name="LSTM_Model")
lstm_model.add(Embedding(vocab_size, embd_len, input_length=max_words))
lstm_model.add(LSTM(128, activation='relu', return_sequences=False))
lstm_model.add(Dense(1, activation='sigmoid'))
# Printing Model Summary
print(lstm_model.summary())
lstm_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

# Training the model
history = lstm_model.fit(x_train, y_train, batch_size=64, epochs=5, verbose=1, validation_data=(x_valid, y_valid))

# Printing model score on test data
print()
print("Simple LSTM score: ", lstm_model.evaluate(x_test, y_test, verbose=1))

Model: "LSTM_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 400, 32)           160000    
                                                                 
 lstm_1 (LSTM)               (None, 128)               82432     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 242,561
Trainable params: 242,561
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5


KeyboardInterrupt: 